## Inserting Example Data

In [23]:
import pymongo
import pandas as pd

client = pymongo.MongoClient("localhost", 27017)
db = client["testdb"]
coll = db["developers"]

mylist = [
    { "devId": "001", "name": "Thomas", "age": 36, "country": "Germany" },
    { "devId": "002", "name": "John", "age": 44, "country": "Scotland" },
    { "devId": "003", "name": "Lothar", "age": 26, "country": "Germany" },
]
coll.insert_many(mylist)

documents = coll.find()

for document in documents:
    print(document)

{'_id': ObjectId('62ac6d911f3a190a2acad1a0'), 'devId': '001', 'name': 'Thomas', 'age': 36, 'country': 'Germany'}
{'_id': ObjectId('62ac6d911f3a190a2acad1a1'), 'devId': '002', 'name': 'John', 'age': 44, 'country': 'Scotland'}
{'_id': ObjectId('62ac6d911f3a190a2acad1a2'), 'devId': '003', 'name': 'Lothar', 'age': 26, 'country': 'Germany'}


## Aggregation Pipeline 1

In [24]:
pipeline = [
    {
        "$match": {
            "age": {"$gte": 30}
        }
    }, 
    {
        "$sort": {
            "age": pymongo.ASCENDING
        }
    },
]

results = coll.aggregate(pipeline)
df_results = pd.DataFrame(list(results))
df_results

,_id,devId,name,age,country
0,62ac6d911f3a190a2acad1a0,001,Thomas,36,Germany
1,62ac6d911f3a190a2acad1a1,002,John,44,Scotland


## Aggregation Pipeline 2

In [25]:
pipeline = [
    {
        "$match": {
            "country": "Germany"
        }
    }, 
    {
        "$sort": {
            "age": pymongo.ASCENDING
        }
    },
    {
        "$out": {
            "db": "testdb", 
            "coll": "developers_ger" 
        }
    }
]

coll.aggregate(pipeline)

client = pymongo.MongoClient("localhost", 27017)
db = client["testdb"]
coll_new = db["developers_ger"]
cursor = coll_new.find()
df_devs_ger = pd.DataFrame(list(cursor))
df_devs_ger

,_id,devId,name,age,country
0,62ac6d911f3a190a2acad1a2,003,Lothar,26,Germany
1,62ac6d911f3a190a2acad1a0,001,Thomas,36,Germany


## Aggregation Pipeline 3

In [26]:
pipeline = [
    {
        "$group": {
            "_id": "$country", 
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {
         "count": pymongo.DESCENDING
        }
    },
]

results = coll.aggregate(pipeline)
df_results = pd.DataFrame(list(results))
df_results

,_id,count
0,Germany,2
1,Scotland,1
